In [1]:
import pandas as pd
import nltk
import os
from collections import Counter
import string
import itertools
import re
import json
import subprocess

In [2]:
corpus_path = '..' + os.sep + 'Corpus-spell-AP88'

In [3]:
corpus = [f'{corpus_path}/ap881229', f'{corpus_path}/ap881230', f'{corpus_path}/ap881231']

In [4]:
tokens = []
for doc in corpus:
    file_text = []
    ff =  open(doc, 'r', encoding='utf-8')
    in_text_block = False
    lines = ff.readlines()
    for line in lines:
        if line.startswith('<TEXT>'):
            in_text_block = True
            
        elif line.startswith('</TEXT>'):
            in_text_block = False
        elif in_text_block == True:
            line_fixed = line.translate(str.maketrans('', '', """!"#$%&()*+,./:;<=>?@[\]^_`{|}~"""))
            line_tokens = line_fixed.split()
            for tok in line_tokens:
                if tok.isalpha() and tok.islower():
                    tokens.append(tok)
        else:
                pass
fw = open(f'corpus_tokens.txt', 'w', encoding='utf-8')
for c in set(tokens):
    fw.write(f'{c.strip()}\n')
fw.close()


In [5]:
pairs = [''.join([pair[0], pair[1]]) for pair in itertools.combinations(string.ascii_lowercase,2)]
pairs.extend([''.join([a[0], a[1]]) for a in zip(string.ascii_lowercase, string.ascii_lowercase) ])
pairs.sort()

In [6]:
long_string = ' '.join(tokens)

In [7]:
pair_regexes = [(p, re.compile(p)) for p in pairs]

In [8]:
pair_counts = [(p[0], max(1,len(p[1].findall(long_string)))) for p in pair_regexes]

In [9]:
pair_counts_dict = dict(pair_counts)
with open('letter_pair_counts', 'w') as f:
    f.write(json.dumps(pair_counts_dict, indent=4))

In [10]:
#now execute these in bash
# cat corpus_tokens.txt | aspell -d en_US list > incorrect.txt
# cat incorrect.txt | aspell -d en_US -a > suggestions.txt